In [4]:
# packages imported
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# import data
# file_name 檔案路徑
file_name = "original_data/training_data.csv"
data = pd.read_csv(file_name, header = 0)
train = pd.DataFrame(data)
file_name_test = 'original_data/test_data.csv'
data = pd.read_csv(file_name_test, header = 0)
test = pd.DataFrame(data)

X_test = train['stars'].tolist()

In [5]:
# 將train data按照星數分為五類
# cate_* 裡面放的是該類全句子斷句結果
cate_1 = []
cate_2 = []
cate_3 = []
cate_4 = []
cate_5 = []

vectorizer = CountVectorizer()
analyze = vectorizer.build_analyzer()

for i in range(len(X_test)):
    if(X_test[i] == 1):
        cate_1.append(analyze(train['text'][i]))
    elif(X_test[i] == 2):
        cate_2.append(analyze(train['text'][i]))
    elif(X_test[i] == 3):
        cate_3.append(analyze(train['text'][i]))
    elif(X_test[i] == 4):
        cate_4.append(analyze(train['text'][i]))
    elif(X_test[i] == 5):
        cate_5.append(analyze(train['text'][i]))
           

In [6]:
# 由於cate_*中包含許多list
# 用以下code將list給unlist 
from itertools import chain
unlist_cate_1 = list(chain.from_iterable(cate_1))
unlist_cate_2 = list(chain.from_iterable(cate_2))
unlist_cate_3 = list(chain.from_iterable(cate_3))
unlist_cate_4 = list(chain.from_iterable(cate_4))
unlist_cate_5 = list(chain.from_iterable(cate_5))

# 將unlist之後的cate_*取dict 排序出現次數
dict_cate_1 = {i:unlist_cate_1.count(i) for i in unlist_cate_1}
dict_cate_2 = {i:unlist_cate_2.count(i) for i in unlist_cate_2}
dict_cate_3 = {i:unlist_cate_3.count(i) for i in unlist_cate_3}
dict_cate_4 = {i:unlist_cate_4.count(i) for i in unlist_cate_4}
dict_cate_5 = {i:unlist_cate_5.count(i) for i in unlist_cate_5}


In [7]:
# 此函數輸入 top_num (想要前幾名) 以及 my_dict (想排序的dict集)
# 最後回傳該dict集中排名前top_num的單詞
def find_top(top_num, my_dict):
    counts = list(my_dict.values())
    top_count_pos = []
    counts_sorted = sorted(counts, reverse = True)
    top_num_counts = counts_sorted[top_num]
    booling_array = (np.asarray(counts_sorted) >= top_num_counts)
    
    from itertools import compress
    keys = list(my_dict.keys())
    ans = list(compress(keys, booling_array.tolist()))
    return(ans)
    

In [9]:
# 上述函數帶值
top_num = 300
top_cate_1 = find_top(top_num, dict_cate_1)
top_cate_2 = find_top(top_num, dict_cate_2)
top_cate_3 = find_top(top_num, dict_cate_3)
top_cate_4 = find_top(top_num, dict_cate_4)
top_cate_5 = find_top(top_num, dict_cate_5)

# 合併五類
# unlist後取dict看重複次數
top_total = [top_cate_1, top_cate_2, top_cate_3, top_cate_4, top_cate_5]
top_total = list(chain.from_iterable(top_total))
dict_top_total = {i:top_total.count(i) for i in top_total}

# 若重複次數 高於5 則該單詞對五類都重要
# 沒判斷性  故刪除
keys = list(dict_top_total.keys())
values = list(dict_top_total.values())
var_left = []

for i in range(len(keys)):
    if values[i] <= 4:
        var_left.append(keys[i])

print(len(var_left))

960


In [11]:
# 利用tfidf package斷詞以及找出不重複單詞
train_freq = vectorizer.fit_transform(train['text'])
train_word = vectorizer.get_feature_names()
transformer = TfidfTransformer()
train_tfidf = transformer.fit_transform(train_freq)

# tfidf 矩陣
train_tfidf.toarray()
# 總出現次數
train_freq.toarray()
# features
train_word

In [12]:
# 將篩選出的參數對比原先次數矩陣
# 只從次數矩陣中挑出該參數的重複次數
df_total_freq = pd.DataFrame(train_freq.toarray())
var_left_index = []

for i in range(len(var_left)):
    var_left_index.append(train_word.index(var_left[i]))
    
df_var_left_freq = df_total_freq.iloc[:,var_left_index]
df_var_left_freq.columns = var_left
df_var_left_freq['rated'] = X_test

df_var_left_freq.shape

C:\Users\kami\Anaconda3\envs\data_sciense\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


(7997, 961)

In [82]:
#將上述結果輸出
df_var_left_freq.to_csv('train_var_left_freq.csv', index = False)

In [ ]:
#接 R code

In [41]:
import mord
from sklearn import linear_model, preprocessing

# 讀入 R 處理後的資料
file_name = "train_var_after_logistic_left_freq.csv"
data = pd.read_csv(file_name, header = 0)
train_logistic_var = pd.DataFrame(data)

X = train_logistic_var.iloc[:,0:428]
y = train_logistic_var['rated']

# 配模型
clf = mord.LogisticAT(alpha = 1.)
clf.fit(X,y)

LogisticAT(alpha=1.0, max_iter=1000, verbose=0)

In [46]:
from itertools import repeat

# 將test資料做轉換
# 同樣將test資料做斷詞 得到各單詞重複次數矩陣
# 對比該重複次數矩陣以及 R 篩選過後的單詞
# 只保留篩選後單詞的重複次數矩陣
# 若是重複次數矩陣中無該單詞  則新增該行 並填滿 0
test_text = test['text']

vectorizer_test = CountVectorizer()
vectorizer_test.fit(test_text)
test_freq = pd.DataFrame(vectorizer_test.fit_transform(test_text).toarray())
test_word = vectorizer_test.get_feature_names()

X_colnames = list(X.columns.values)

test_left_var = pd.DataFrame()

for i in range(len(X_colnames)):
    cur_name = X_colnames[i]
    
    if cur_name in test_word:
        cur_pos = test_word.index(cur_name)
        test_left_var[cur_name] = test_freq.iloc[:, cur_pos]
    else:
        test_left_var[cur_name] = pd.Series(list(repeat(0, 7997)))

In [51]:
# 預測並輸出
test_pred = clf.predict(test_left_var)
review_id = test['review_id']
out = pd.DataFrame()
out[0] = review_id
out[1] = test_pred

out.to_csv('out.csv', header = False, index = False)